# Thermo-desorption spectrum

This validation case is a thermo-desorption spectrum measurement perfomed by Ogorodnikova et al. {cite}`ogorodnikova_deuterium_2003`.

Deuterium ions at 200 eV were implanted in a 0.5 mm thick sample of high purity tungsten foil (PCW).

The ion beam was turned on for an implanted fluence of $1.0 \times 10^{22} \ \mathrm{D \ m^{-2}}$

The trapping parameters are taken from Table 1 for both the intrinsic traps and the extrinsic, ion-induced traps.

## FESTIM code

In [13]:
import festim as F
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

flux_model = F.Simulation()

vertices = np.concatenate([
    np.linspace(0, 30e-9, num=200),
    np.linspace(30e-9, 3e-6, num=300),
    np.linspace(3e-6, 20e-6, num=200),
])

flux_model.mesh = F.MeshFromVertices(vertices)
# Material Setup, only W
#　Data from Table 1
tungsten = F.Material(
    id=1,
    D_0=4.1e-07,  # m2/s
    E_D=0.39,  # eV
)

flux_model.materials = tungsten
import sympy as sp

imp_fluence = 1e22
flux_strength = 2.5e19 # beam strength from paper

imp_time = imp_fluence/flux_strength # s

ion_flux = sp.Piecewise((flux_strength, F.t <= imp_time), (0, True))

source_term = F.ImplantationFlux(
    flux=ion_flux,  # H/m2/s
    imp_depth=4.5e-9,  # m
    width=2.5e-9,  # m
    volume=1
)

flux_model.sources = [source_term]
#trap settings
w_atom_density = 6.3e28  # atom/m3

trap_1 = F.Trap(
        k_0=4.1e-7/(1.1e-10**2*6*w_atom_density),
        E_k=0.39,
        p_0=1e13,
        E_p=0.87,
        density=1.3e-3*w_atom_density,
        materials=tungsten
    )

trap_2 = F.Trap(
        k_0=4.1e-7/(1.1e-10**2*6*w_atom_density),
        E_k=0.39,
        p_0=1e13,
        E_p=1.0,
        density=4e-4*w_atom_density,
        materials=tungsten
    )

center = 4.5e-9
width = 2.5e-9
distribution = 1/(width*(2*sp.pi)**0.5) * sp.exp(-0.5*((F.x-center)/width)**2)

trap_3 = F.ExtrinsicTrap(
        k_0=4.1e-7/(1.1e-10**2*6*w_atom_density),
        E_k=0.39,
        p_0=1e13,
        E_p=1.5,
        phi_0=ion_flux,
        n_amax=1e-01*w_atom_density, f_a=distribution, eta_a=6e-4,
        n_bmax=1e-02*w_atom_density, f_b=sp.Piecewise((1e6, F.x < 1e-6), (0, True)), eta_b=2e-4,
        materials=tungsten
    )


center = 4.5e-9
width = 2.5e-9
distribution = 1/(width*(2*sp.pi)**0.5) * sp.exp(-0.5*((F.x-center)/width)**2)

flux_model.traps = [trap_1, trap_2, trap_3]
#boundary conditions
flux_model.boundary_conditions = [
    F.DirichletBC(surfaces=[1, 2], value=0, field=0)
]
implantation_temp = 293  # K
temperature_ramp = 8  # K/s

start_tds = imp_time + 300  # s

flux_model.T = F.Temperature(
    value=sp.Piecewise(
        (implantation_temp, F.t < start_tds),
        (implantation_temp + temperature_ramp*(F.t-start_tds), True))
)
min_temp, max_temp = 200, 900

flux_model.dt = F.Stepsize(
    initial_value=0.5,
    stepsize_change_ratio=1.1,
    max_stepsize=lambda t: 0.5 if t > start_tds else None,
    dt_min=1e-05,
    milestones=[start_tds],
)

flux_model.settings = F.Settings(
    absolute_tolerance=1e10,
    relative_tolerance=1e-09,
    final_time=start_tds + (max_temp - implantation_temp)/temperature_ramp # time to reach max temp
)
derived_quantities = F.DerivedQuantities(
    [
        F.TotalVolume("solute", volume=1),
        F.TotalVolume("retention", volume=1),
        F.TotalVolume("1", volume=1),
        F.TotalVolume("2", volume=1),
        F.TotalVolume("3", volume=1),
        F.HydrogenFlux(surface=1),
        F.HydrogenFlux(surface=2)
    ],
    # filename="tds/derived_quantities.csv"  # optional set a filename to export the data to csv
)

flux_model.exports = [derived_quantities]

flux_model.initialise()
flux_model.run()

## Comparison with experimental data

The results produced by FESTIM are in good agreement with the experimental data.

In [48]:
t = derived_quantities.t
flux_left = derived_quantities.filter(fields="solute", surfaces=1).data
flux_right = derived_quantities.filter(fields="solute", surfaces=2).data

flux_total = -np.array(flux_left) - np.array(flux_right)
#plotting simulation data
temp = [implantation_temp + 8*(t - start_tds) for t in t]
plt.plot(temp, flux_total, linewidth=3, label = "FESTIM")

#plotting original data
experimental_tds = np.genfromtxt("ogorodnikova-original.csv", delimiter=",")
experimental_temp = experimental_tds[:, 0]
experimental_flux = experimental_tds[:, 1]
plt.plot(experimental_temp, experimental_flux, linewidth=3, color="green", label="original")

plt.legend()
plt.xlim(min_temp, max_temp)
plt.ylim(bottom=0, top=1e19)
plt.ylabel(r"Desorption flux (m$^{-2}$ s$^{-1}$)")
plt.xlabel(r"Temperature (K)")

plt.show()

```{note}
The experimental data was taken from Figure 5 of the original experiment paper {cite}`ogorodnikova_deuterium_2003` using [WebPlotDigitizer](https://automeris.io/)
```